In [1]:
import os
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import mysql.connector as sql
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [11]:
# make a connection to the database
dwh = sql.connect(
    host="127.0.0.1",   # ip-adres
    port=3306,   # poortnummer
    user=os.getenv("USER"),   # username
    password=os.getenv("PASSWORD"),   # wachtwoord
    database="AirFaresDWH"   # naam van de datawarehouse in mysql workbench
    # database="groep8dep"   # naam van de OLTP database in mysql workbench
)

# check if the connection succeeded
print(dwh)

,flightfare_key,flight_key,airline_key,depatureAirportKey,arrivalAirportKey,scrapeDateKey,departureDateKey,arrivalDateKey,availableSeats,price
0,131071,5545,1,7,9,20230423,20230524,20230524,0,90.99


In [25]:
# create query
query = """
select flightfare_key, flight_key, airline_key, depatureAirportKey, arrivalAirportKey, scrapeDateKey, de.date as scrapeDate, departureDateKey, dd.date as departureDate, arrivalDateKey, availableSeats, price from factflightfare ff
join dimdate dd on dd.date_key = ff.departureDateKey
join dimdate de on de.date_key = ff.scrapeDateKey;
"""

# create the dataframe while reading query
df = pd.read_sql(query, dwh)

# show all columns
pd.options.display.max_columns = None

# show the df
df

,flightfare_key,flight_key,airline_key,depatureAirportKey,arrivalAirportKey,scrapeDateKey,scrapeDate,departureDateKey,departureDate,arrivalDateKey,availableSeats,price
0,1,1153,1,7,9,20230420,2023-04-20,20230523,2023-05-23,20230523,2,87.79
1,2,982,1,7,8,20230420,2023-04-20,20230623,2023-06-23,20230623,1,77.74
2,3,726,1,7,4,20230420,2023-04-20,20230627,2023-06-27,20230627,-1,37.77
3,4,510,1,7,2,20230420,2023-04-20,20230702,2023-07-02,20230702,-1,134.12
4,5,1590,1,7,14,20230420,2023-04-20,20230717,2023-07-17,20230717,-1,181.49
...,...,...,...,...,...,...,...,...,...,...,...,...
114260,120528,2879,4,3,2,20230430,2023-04-30,20230711,2023-07-11,20230711,-1,219.99
114261,120529,4015,4,5,12,20230430,2023-04-30,20230922,2023-09-22,20230922,-1,149.99
114262,120530,2933,4,3,2,20230430,2023-04-30,20230924,2023-09-24,20230924,-1,129.99
114263,120531,4619,4,5,17,20230430,2023-04-30,20231004,2023-10-04,20231004,-1,204.99


## Vraag 12: Hoeveel dagen voor vertrek is een vlucht volgeboekt? Zijn er bestemmingen waarbij de vluchten bijna volgeboekt zijn?

In [29]:
# 1ste vraag
# datum van vandaag of scrape date (VRAGEN)
# vertrek datum
# verschil tussen de 2 als er geen zitplaatsen vrij zijn

def calc_date_diff(row):
    if row['availableSeats'] == 0:
        today = dt.date.today()
        return (row['departureDate'] - today).days # row['scrapeDate']
    else:
        return "nog niet volzet"

df['aantalDagenVoorVertrekVolzet'] = df.apply(calc_date_diff, axis=1)
df


,flightfare_key,flight_key,airline_key,depatureAirportKey,arrivalAirportKey,scrapeDateKey,scrapeDate,departureDateKey,departureDate,arrivalDateKey,availableSeats,price,aantalDagenVoorVertrekVolzet
0,1,1153,1,7,9,20230420,2023-04-20,20230523,2023-05-23,20230523,2,87.79,nog niet volzet
1,2,982,1,7,8,20230420,2023-04-20,20230623,2023-06-23,20230623,1,77.74,nog niet volzet
2,3,726,1,7,4,20230420,2023-04-20,20230627,2023-06-27,20230627,-1,37.77,nog niet volzet
3,4,510,1,7,2,20230420,2023-04-20,20230702,2023-07-02,20230702,-1,134.12,nog niet volzet
4,5,1590,1,7,14,20230420,2023-04-20,20230717,2023-07-17,20230717,-1,181.49,nog niet volzet
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114260,120528,2879,4,3,2,20230430,2023-04-30,20230711,2023-07-11,20230711,-1,219.99,nog niet volzet
114261,120529,4015,4,5,12,20230430,2023-04-30,20230922,2023-09-22,20230922,-1,149.99,nog niet volzet
114262,120530,2933,4,3,2,20230430,2023-04-30,20230924,2023-09-24,20230924,-1,129.99,nog niet volzet
114263,120531,4619,4,5,17,20230430,2023-04-30,20231004,2023-10-04,20231004,-1,204.99,nog niet volzet


In [ ]:
# 2de vraag
# Zijn er bestemmingen waarbij de vluchten bijna volgeboekt zijn?
# powerbi (vraag12p2)
